In [2]:
# install beautifulsoup
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [3]:
# import beautifulsoup library
from bs4 import BeautifulSoup as bs
import requests

In [4]:
# get the table data from wiki page
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)
soup = bs(response.text, 'html.parser')
print(soup.title)
#soup.find('table').find_all('tr')[1].find_all('td')[0].string

In [41]:
# create a empty dataframe with three columns as below
import pandas as pd
column_name =['PostalCode','Borough','Neighborhood']
neighborhoods = pd.DataFrame(columns = column_name)
neighborhoods

,PostalCode,Borough,Neighborhood


In [42]:
#  get the table data from wiki page and store it to new data frame
listcode=[]
for row in soup.find('table',{'class': 'wikitable sortable'}).find_all('tr'):
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    
    if len(cols)>0:
        #print (cols)
        listcode.append([ele for ele in cols if ele])

df = pd.DataFrame(listcode)
#print (df)
neighborhoods[['PostalCode','Borough','Neighborhood']]=pd.DataFrame(df.values)

In [43]:
# display new dataframe 
neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [44]:
# remove records which with a borough as 'Not assigned' 
neighborhoods = neighborhoods[neighborhoods['Borough'] != 'Not assigned']
neighborhoods.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [45]:
# combine neigborhood column data where borough are same with a comma
neighborhoods['Neighborhood']=neighborhoods.groupby('PostalCode')['Neighborhood'].transform(lambda x: ','.join(x))
neighborhoods= neighborhoods[['PostalCode','Borough','Neighborhood']].drop_duplicates()
neighborhoods.reset_index(inplace =True, drop=True)
neighborhoods.head()

C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Not assigned


In [55]:
# select records which has a borough but a Not assigned neighborhood, 
# then the neighborhood will be the same as the borough

for i, rw in neighborhoods.iterrows():
    #print (rw)
    if rw['Neighborhood'] == 'Not assigned':
        rw['Neighborhood']=rw['Borough']

In [57]:
neighborhoods.head(5)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [61]:
# Check the datafrome shape
neighborhoods.shape

(103, 3)